### Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import time
from ETGEMs_function_ETG import *
import pandas as pd
from ppmcot import *

In [2]:
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

### Data initialization

In [3]:
#Get Reaction G0 from local file_original
reaction_g0_file_original1 = './reaction_g0_ETC_butanoic_acid.txt'
reaction_g0_file_original0 = './reaction_g0_ETC_butanol.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original1 = './metabolites_lnC_ETC_butanoic_acid.txt'
metabolites_lnC_file_original0 = './metabolites_lnC_ETC_butanol.txt'
#Get Model from local file
model_file_original1 = './iML1515_butanoic_acid.xml'
model_file_original0 = './iML1515_butanol.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_ETC_butanoic_acid.csv'
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_ETC_butanol.csv'

In [4]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

### Strain reconstruction

In [5]:
#gene deletion
model0['ub_list']['EX_but_e'] = 0
model0['lb_list']['PPC'] = 0
model0['lb_list']['LDH_D'] = 0
model0['lb_list']['ACALD'] = 0
model0['lb_list']['PTAr'] = 0
model0['ub_list']['PPC'] = 0
model0['ub_list']['LDH_D'] = 0
model0['ub_list']['ACALD'] = 0
model0['ub_list']['PTAr'] = 0
model0['lb_list']['ACACT1r'] = 0
model0['lb_list']['HACD1'] = 0
model0['lb_list']['ECOAH1'] = 0
model0['ub_list']['ACACT1r'] = 0
model0['ub_list']['HACD1'] = 0
model0['ub_list']['ECOAH1'] = 0

In [6]:
model1['ub_list']['BUTCT'] = 0
model1['ub_list']['EX_ac_e'] = 0
model1['lb_list']['PPC'] = 0
model1['lb_list']['LDH_D'] = 0
model1['lb_list']['ACALD'] = 0
model1['ub_list']['PPC'] = 0
model1['ub_list']['LDH_D'] = 0
model1['ub_list']['ACALD'] = 0
model1['ub_list']['PTAr'] = 0

### Predict the productions (g/L) under different inoculum ratios

In [8]:
#If you want to simulate the production with the unit of g/L, the real molar masses of exchange metabolites and carbon sources should be input, and the concentrations of exchange metabolites and carbon sources should be input with the unit of g/L
#If you want to simulate the production with the unit of mM, the molar masses of exchange metabolites and carbon sources should be input with the same value, and the concentrations of exchange metabolites and carbon sources should be input with the unit of mM

#model_list: the list of models calculated by the function of Get_Concretemodel_Need_Data
#target_list: the double list consist of the list of target reactions of each model, where the first reaction of each target reaction list is biomass biosynthesis reaction and the second is the reaction whose flux need to be maximized or minimized
#growth_list: the list of minimize growth rates in the function of MDF_Calculation
#growth_ratio: the list of ratios of lower bounds of growth rates to maximum growth rates
#parameter_list: the list of the total number of enzyme ratios, the upper bounds for substrate input reaction fluxes and the maximum values minus the minimum values of reaction thermodynamic driving forces
#substrate_environment: the list of the carbon source uptake reactions and the initial carbon source concentrations in environment
#metabolites_monitored: the list of metabolites delivered by the strains and the product
#metabolites_mass: the dictionary whose keys are the metabolites in metabolites_monitored and values are the molar masses of the metabolites
#concentrations_initial: the dictionary whose keys are the metabolites in metabolites_monitored and values are the initial concentrations of the metabolites in environment
#reactions_monitored: the list of dictionaries whose keys are the reactions of the metabolite uptake or release in metabolites_monitored and values are the ratios of lower bounds of the reaction fluxes to the max reaction fluxes
#substrate_mass: the list of the molar masses of the carbon sources
#t: the number of the time points for mono-cultivation
#culture_condition: the list of the inoculum ratio of the straints and time points for co-cultivation
#map_coefficient: the list of mapping constants of the instantaneous fluxes corresponding to the mean fluxes for the mono strains
#constraints: the constraints used in the simulation, where 'kinetics and thermodynamics' consist of FBA, kinetic and thermodynamic constraints, 'kinetics' consist of FBA and kinetic constraints, 'thermodynamics' consist of FBA and thermodynamic constraints and 'FBA' consist of FBA constraint

model_list=[model0, model1]
target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']]
growth_list=[0.2, 0.2]
growth_ratio=[0.24, 0.47]
parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]]
substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]]
metabolites_monitored=['ac_e', 'but_e', 'btoh_e']
metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}
concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}
reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}]
substrate_mass=[180,180]
t=60
culture_condition=[[1/4, 3/4], 60]
map_coefficient=[10/(10+10-7.8)*2, 12/(12+0.7)*2]
constraints='kinetics and thermodynamics'

ppmcot(model_list, target_list, growth_list,growth_ratio, parameter_list, substrate_environment,  metabolites_monitored, metabolites_mass, concentrations_initial, reactions_monitored, substrate_mass, t, culture_condition, map_coefficient, constraints)

After 1 time point, the concentration of ac_e is 0.001806004447953394 g/L or mM
After 1 time point, the concentration of but_e is 0.15224859176936612 g/L or mM
After 1 time point, the concentration of btoh_e is 0 g/L or mM
After 2 time point, the concentration of ac_e is 0.005573751093129484 g/L or mM
After 2 time point, the concentration of but_e is 0.2998963341650177 g/L or mM
After 2 time point, the concentration of btoh_e is 0.003904145668461699 g/L or mM
After 3 time point, the concentration of ac_e is 0.012353696833816367 g/L or mM
After 3 time point, the concentration of but_e is 0.44312198260294755 g/L or mM
After 3 time point, the concentration of btoh_e is 0.01159445636632043 g/L or mM
After 4 time point, the concentration of ac_e is 0.021960459534920644 g/L or mM
After 4 time point, the concentration of but_e is 0.5821195267316641 g/L or mM
After 4 time point, the concentration of btoh_e is 0.02295753532077184 g/L or mM
After 5 time point, the concentration of ac_e is 0.0342

After 36 time point, the concentration of ac_e is 1.0846724867588935 g/L or mM
After 36 time point, the concentration of but_e is 3.5826710862235087 g/L or mM
After 36 time point, the concentration of btoh_e is 1.8448691839288898 g/L or mM
After 37 time point, the concentration of ac_e is 1.1145767726837792 g/L or mM
After 37 time point, the concentration of but_e is 3.648085655707175 g/L or mM
After 37 time point, the concentration of btoh_e is 1.9367404429024457 g/L or mM
After 38 time point, the concentration of ac_e is 1.144930481086192 g/L or mM
After 38 time point, the concentration of but_e is 3.7121025020222715 g/L or mM
After 38 time point, the concentration of btoh_e is 2.030289142732012 g/L or mM
After 39 time point, the concentration of ac_e is 1.1756908646694524 g/L or mM
After 39 time point, the concentration of but_e is 3.77477239492594 g/L or mM
After 39 time point, the concentration of btoh_e is 2.1254794412807856 g/L or mM
After 40 time point, the concentration of ac_

{'ac_e': 1.6660082071520985,
 'but_e': 5.164282986712774,
 'btoh_e': 4.161629509541719}

In [8]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.24, 0.47], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[1/3, 2/3], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='kinetics and thermodynamics')

After 1 time point, the concentration of ac_e is 0.002380503684605627 g/L or mM
After 1 time point, the concentration of but_e is 0.1337864308118422 g/L or mM
After 1 time point, the concentration of btoh_e is 0 g/L or mM
After 2 time point, the concentration of ac_e is 0.005898566691725631 g/L or mM
After 2 time point, the concentration of but_e is 0.26359157017751417 g/L or mM
After 2 time point, the concentration of btoh_e is 0.003391533432668611 g/L or mM
After 3 time point, the concentration of ac_e is 0.012028383372650801 g/L or mM
After 3 time point, the concentration of but_e is 0.38955738344829316 g/L or mM
After 3 time point, the concentration of btoh_e is 0.010073673148944971 g/L or mM
After 4 time point, the concentration of ac_e is 0.02061094333380299 g/L or mM
After 4 time point, the concentration of but_e is 0.5118503321060681 g/L or mM
After 4 time point, the concentration of btoh_e is 0.019949090871733056 g/L or mM
After 5 time point, the concentration of ac_e is 0.031

After 36 time point, the concentration of ac_e is 0.9596008010170392 g/L or mM
After 36 time point, the concentration of but_e is 3.1606367788672203 g/L or mM
After 36 time point, the concentration of btoh_e is 1.6073712027100968 g/L or mM
After 37 time point, the concentration of ac_e is 0.9972318857779925 g/L or mM
After 37 time point, the concentration of but_e is 3.217891260563427 g/L or mM
After 37 time point, the concentration of btoh_e is 1.6874944651291812 g/L or mM
After 38 time point, the concentration of ac_e is 1.0349069207862571 g/L or mM
After 38 time point, the concentration of but_e is 3.274153905470678 g/L or mM
After 38 time point, the concentration of btoh_e is 1.7690691489492583 g/L or mM
After 39 time point, the concentration of ac_e is 1.0726041963745538 g/L or mM
After 39 time point, the concentration of but_e is 3.3294551073422576 g/L or mM
After 39 time point, the concentration of btoh_e is 1.8520701107562398 g/L or mM
After 40 time point, the concentration of 

{'ac_e': 1.8421233326744342,
 'but_e': 4.310837822487642,
 'btoh_e': 3.8869907951349076}

In [9]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.24, 0.47], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[1/2, 1/2], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='kinetics and thermodynamics')

After 1 time point, the concentration of ac_e is 0.003488248788911747 g/L or mM
After 1 time point, the concentration of but_e is 0.09802134696749015 g/L or mM
After 1 time point, the concentration of btoh_e is 0 g/L or mM
After 2 time point, the concentration of ac_e is 0.007594571936797518 g/L or mM
After 2 time point, the concentration of but_e is 0.1932262767809714 g/L or mM
After 2 time point, the concentration of btoh_e is 0 g/L or mM
After 3 time point, the concentration of ac_e is 0.012478956555244964 g/L or mM
After 3 time point, the concentration of but_e is 0.28570934941656567 g/L or mM
After 3 time point, the concentration of btoh_e is 0.004785172030388767 g/L or mM
After 4 time point, the concentration of ac_e is 0.019111225044526434 g/L or mM
After 4 time point, the concentration of but_e is 0.37556544377947565 g/L or mM
After 4 time point, the concentration of btoh_e is 0.011860650637258899 g/L or mM
After 5 time point, the concentration of ac_e is 0.0273862416499001 g/L

After 36 time point, the concentration of ac_e is 0.7131625452437549 g/L or mM
After 36 time point, the concentration of but_e is 2.338390354991215 g/L or mM
After 36 time point, the concentration of btoh_e is 1.156053944995123 g/L or mM
After 37 time point, the concentration of ac_e is 0.7410308261973558 g/L or mM
After 37 time point, the concentration of but_e is 2.3810909454181273 g/L or mM
After 37 time point, the concentration of btoh_e is 1.213963253756072 g/L or mM
After 38 time point, the concentration of ac_e is 0.7689455855656118 g/L or mM
After 38 time point, the concentration of but_e is 2.4230526988116132 g/L or mM
After 38 time point, the concentration of btoh_e is 1.2729300256860885 g/L or mM
After 39 time point, the concentration of ac_e is 0.7968904711912282 g/L or mM
After 39 time point, the concentration of but_e is 2.46429820200848 g/L or mM
After 39 time point, the concentration of btoh_e is 1.3329359637793712 g/L or mM
After 40 time point, the concentration of ac_

{'ac_e': 1.3699798870071955,
 'but_e': 3.196410177201423,
 'btoh_e': 2.8057656070550445}

In [10]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.24, 0.47], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[2/3, 1/3], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='kinetics and thermodynamics')

After 1 time point, the concentration of ac_e is 0.004540989401220071 g/L or mM
After 1 time point, the concentration of but_e is 0.06380191388406575 g/L or mM
After 1 time point, the concentration of btoh_e is 0 g/L or mM
After 2 time point, the concentration of ac_e is 0.009833747791866273 g/L or mM
After 2 time point, the concentration of but_e is 0.12585613190703984 g/L or mM
After 2 time point, the concentration of btoh_e is 0 g/L or mM
After 3 time point, the concentration of ac_e is 0.015025158457173976 g/L or mM
After 3 time point, the concentration of but_e is 0.18622677617130612 g/L or mM
After 3 time point, the concentration of btoh_e is 0 g/L or mM
After 4 time point, the concentration of ac_e is 0.019816533744977435 g/L or mM
After 4 time point, the concentration of but_e is 0.2449603341794243 g/L or mM
After 4 time point, the concentration of btoh_e is 0.004502749581944464 g/L or mM
After 5 time point, the concentration of ac_e is 0.025643659533650512 g/L or mM
After 5 ti

After 36 time point, the concentration of ac_e is 0.48054254359818127 g/L or mM
After 36 time point, the concentration of but_e is 1.5401621035956226 g/L or mM
After 36 time point, the concentration of btoh_e is 0.7379873168699306 g/L or mM
After 37 time point, the concentration of ac_e is 0.49900014986386104 g/L or mM
After 37 time point, the concentration of but_e is 1.568587025955714 g/L or mM
After 37 time point, the concentration of btoh_e is 0.7752266677875009 g/L or mM
After 38 time point, the concentration of ac_e is 0.5174946826204364 g/L or mM
After 38 time point, the concentration of but_e is 1.5965300536590907 g/L or mM
After 38 time point, the concentration of btoh_e is 0.8131533006840932 g/L or mM
After 39 time point, the concentration of ac_e is 0.5360154793486961 g/L or mM
After 39 time point, the concentration of but_e is 1.6240056930653064 g/L or mM
After 39 time point, the concentration of btoh_e is 0.8517555639003221 g/L or mM
After 40 time point, the concentration 

{'ac_e': 0.9173729271782485,
 'but_e': 2.1132793219294514,
 'btoh_e': 1.801020611712008}